In [6]:
from benchmark import run_benchmark
from benchmark import print_results
import unit_preference_methods
from splitting_methods import first_choice
from splitting_methods import random_choice
from splitting_methods import two_clause_choice


In [7]:
results = run_benchmark(100, 3, 6, 0.2, 5, 1,
                        [unit_preference_methods.random_choice],
                        [two_clause_choice])

In [9]:
print_results(results)

----RESULTS----
Ratio: 3.0
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.0456186549999984, 101, True)
Ratio: 3.2
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.06313331400087918, 263, True)
Ratio: 3.4000000000000004
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.40522097799839685, 2031, True)
Ratio: 3.6000000000000005
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.04943027699846425, 225, True)
Ratio: 3.8000000000000007
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.1460570090021065, 665, True)
Ratio: 4.000000000000001
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (-1, None, True)
Ratio: 4.200000000000001
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.10796425300213741, 455, True)
Ratio: 4.400000000000001
   Unit Pref: random_choice, Splitting: two_clause_choice:
      (0.4295769480013405, 1505, True)
Ratio: 4.600000000000001
   Unit Pref: ran

In [ ]:
print("Specific result:")
print()